# Описание проекта
Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

Загрузим необходимые для работы библиотеки

In [29]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from scipy.spatial import distance

Загрузим данные и посмотрим общую информацию

In [30]:
data = pd.read_csv('/datasets/insurance.csv')
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


1. Предоставлены данные о 5000 клиентах.
2. Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи. Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [31]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


1. 50/50 представлено женщин и мужчин
2. Средний возраст 31 год. Возраст варьруется от 18 до 65
3. Средняя зарплата - 40 тыс. Зарплата варьируется от 33 до 79 тыс.
4. Общее количество членов семьи варьируется от 0 до 6
5. Страховые выплаты за 5 лет составляют от 0 до 5 раз

## 2. Умножение матриц

Для исследований разделим данные на обучающую и тестовую выборки

In [32]:
X = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

Проверим работу линейной регрессии на первичных данных

In [34]:
def metrics(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print('MSE:', mse)
    print('r2:', r2)
    return mse, r2

In [35]:
mse_original, r2_original = metrics(X_train, X_test, y_train, y_test) 

MSE: 0.13862989572441428
r2: 0.42547785406963123


Создадим произвольную матрицу размером (4, 4)

In [36]:
state = np.random.RandomState(12345)
matrix = state.normal(size=(4, 4))
matrix

array([[-0.20470766,  0.47894334, -0.51943872, -0.5557303 ],
       [ 1.96578057,  1.39340583,  0.09290788,  0.28174615],
       [ 0.76902257,  1.24643474,  1.00718936, -1.29622111],
       [ 0.27499163,  0.22891288,  1.35291684,  0.88642934]])

Найдем у данной матрицы обратную

In [37]:
matrix_inv = np.linalg.inv(matrix)
matrix_inv

array([[-1.31136747,  0.3921804 ,  0.18868055, -0.67088287],
       [ 1.75872714,  0.14106138, -0.17773045,  0.79787127],
       [-0.41702659, -0.22854768,  0.3550602 ,  0.33039819],
       [ 0.58912996,  0.19073027, -0.5545481 ,  0.6259302 ]])

Обратная матрица есть, значит матрица matrix является обратимой

Умножим признаки на матрицу matrix и проверим показатели метрики на новых данных

In [38]:
X_train_conv = X_train.values @ matrix
X_test_conv = X_test.values @ matrix

In [39]:
mse_conv, r2_conv = metrics(X_train_conv, X_test_conv, y_train, y_test)

MSE: 0.13862989572442275
r2: 0.42547785406959604


Значения метрик линейной регрессии не изменилось. Сравним параметры линейной регрессии

In [40]:
def fit(train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w1 = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        w = w1[1:]
        w0 = w1[0]
        return w, w0

Параметры для исходной матрицы

In [41]:
w_clean, w0_clean = fit(X_train, y_train)
w_clean

array([ 4.92432086e-03,  3.51527196e-02, -2.45796619e-07, -1.49140089e-02])

In [42]:
w0_clean

-0.9200267704145323

Параметры для преобразованной матрицы

In [43]:
w_conv, w0_conv = fit(X_train_conv, y_train)
w_conv 

array([ 0.01733416,  0.00171976, -0.01501528,  0.00027074])

In [44]:
w0_conv

-0.9200261702083075

Величина сдвига одинакова. Отличаются векторы весов, следовательно можно записать такое выражение

$$ 
a = a'
$$

$$
Xw+w0 = XPw'+w0
$$

$$
Xw = XPw'
$$

$$
w = Pw'
$$


Обозначения:
- $a$ — предсказание с исходными признаками

- $a'$ — предсказание с преобразованными признаками

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — обратимая матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Следовательно векторы сдвига отличаются на величину обратимой матрицы. Проверим

In [45]:
w_new = matrix @ w_conv
w_new 

array([ 4.92429050e-03,  3.51527195e-02, -2.45804749e-07, -1.49140087e-02])

Все верно. w_clean и w_new совпадают.

Выражение $w=Pw'$ также можно записать в другом виде
$$
w'=P^{-1}w
$$

## 3. Алгоритм преобразования

**Алгоритм**


Для того чтобы зашифровать данные применим умножение признаков на обратимую матрицу

$$
Z = XP
$$

Для расшифровки данных необходимо умножить зашифрованную матрицу признаков на обратную

$$
X = ZP^{-1}
$$

**Обоснование**

Проведем сравнение расчета предсказания с помощью исходной матрицы $a=Xw$ c преобразованной матрицей. Обозначим результат предсказания преобразованной матрицы $a'$, тогда:

<font color='orange'>$$ w'= (Z^T Z)^{-1} Z^T y = ((XP)^T X P)^{-1} (XP)^T y= ((P^T X^T XP)^{-1} (XP)^T y = (X^T X)^{-1} (P^T P)^{-1} (XP)^T y = (X^T X)^{-1} (P^T)^{-1} P^{-1} P^T X^T  y = (X^T X)^{-1} E P^{-1} X^T y = P^{-1} (X^T X)^{-1} X^T y$$

$$ a'= Z w' = XP P^{-1} (X^T X)^{-1} X^T y = X E (X^T X)^{-1} X^T y = X (X^T X)^{-1} X^T y = a$$

Умножение $PP^{-1}$ согласно свойству обратимых матриц равно единичной матрице $E$, а умножение матрицы на единичную равно самой матрице. Следовательно метрика линейной регрессии при данном шифровании не изменится и расшифровка даст исходные данные

## 4. Проверка алгоритма

Создадим алгоритм шифрования и расшифровки данных

In [46]:
class Encoding:
    def code(self, data):
        data_np = data.values
        X_data = data_np[:, :-1]
        y_data = data_np[:, -1:]
        self.matrix = np.random.normal(size=((data.shape[1] - 1), (data.shape[1] - 1)))
        X_data_code = X_data @ self.matrix
        self.code_data = pd.DataFrame(np.concatenate((X_data_code, y_data), axis=1), columns=data.columns)
        return self.code_data
    def decoding(self):
        data_np = self.code_data.values
        X_data = data_np[:, :-1]
        y_data = data_np[:, -1:]
        X_data_decoding = np.around(X_data @ np.linalg.inv(self.matrix))
        data_decoding = pd.DataFrame(np.concatenate((X_data_decoding, y_data), axis=1), columns=self.code_data.columns)
        return data_decoding
        
        

In [47]:
model = Encoding()
data_code = model.code(data)
data_code

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,41370.015875,58684.926232,-70570.142165,-61477.990540,0.0
1,31676.874633,44941.589996,-54059.750678,-47128.235734,1.0
2,17502.015721,24830.517963,-29873.872473,-26051.979664,0.0
3,34795.031817,49358.753224,-59335.505330,-51658.420857,0.0
4,21762.543005,30870.600501,-37132.100888,-32363.549030,0.0
...,...,...,...,...,...
4995,29776.405239,42243.566799,-50793.648126,-44245.063194,0.0
4996,43715.910178,62011.520674,-74558.177816,-64929.735130,0.0
4997,28282.744196,40122.723183,-48235.362744,-42001.190452,0.0
4998,27277.763258,38699.149881,-46526.008980,-40519.474468,0.0


In [48]:
data_decoding = model.decoding()
data_decoding

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1.0,41.0,49600.0,1.0,0.0
1,-0.0,46.0,38000.0,1.0,1.0
2,-0.0,29.0,21000.0,-0.0,0.0
3,-0.0,21.0,41700.0,2.0,0.0
4,1.0,28.0,26100.0,0.0,0.0
...,...,...,...,...,...
4995,-0.0,28.0,35700.0,2.0,0.0
4996,-0.0,34.0,52400.0,1.0,0.0
4997,-0.0,20.0,33900.0,2.0,0.0
4998,1.0,22.0,32700.0,3.0,0.0


Проверим работу линейной регрессии на закодированных данных

In [49]:
X_code = data_code.drop('Страховые выплаты', axis=1)
y_code = data_code['Страховые выплаты']

In [50]:
X_train_code, X_test_code, y_train_code, y_test_code = train_test_split(
    X_code, y_code, test_size=0.25, random_state=42)

In [51]:
mse_code, r2_code = metrics(X_train_code, X_test_code, y_train_code, y_test_code) 

MSE: 0.13862989572455808
r2: 0.42547785406903516


In [52]:
mse_original, r2_original = metrics(X_train, X_test, y_train, y_test) 

MSE: 0.13862989572441428
r2: 0.42547785406963123


### Вывод

Метрики зашифрованных данных и исходных совпадают, следовательно шифровка выполнена верно